# Модуль 9.2 — Information Extraction (IE)

**Цель:** извлекать связи между сущностями (relation extraction) и события (event extraction), использовать LLM для аннотации.

**Что сделаем:**
- relation extraction — кто с кем связан
- event extraction — что произошло, кто участник
- LLM как вспомогательный инструмент для аннотации (zero-shot, few-shot)

## 1. Relation Extraction — классический подход

Relation extraction находит пары (сущность1, тип_связи, сущность2). Например: (Яндекс, является_работодателем, Иванов).

In [ ]:
import spacy

nlp = spacy.load("ru_core_news_sm")

text = "Иван Петров работает в компании Яндекс. Его руководитель — директор Мария Сидорова."
doc = nlp(text)

# 1) Извлекаем сущности (NER)
ents = [(e.text, e.label_) for e in doc.ents]
print("Сущности:", ents)

# 2) Эвристика: ищем шаблоны типа "X работает в Y", "руководитель — Z"
import re
if "работает" in text and "компании" in text:
    m = re.search(r"([А-Яа-яё]+ [А-Яа-яё]+) работает в компании ([А-Яа-яё]+)", text)
    if m:
        print(f"Отношение: ({m.group(1)}, works_at, {m.group(2)})")
if "руководитель" in text:
    m = re.search(r"руководитель — ([А-Яа-яё]+ [А-Яа-яё]+)", text)
    if m:
        print(f"Отношение: (руководитель, is, {m.group(1)})")

## 2. LLM для Relation Extraction

LLM гибко извлекает связи без переобучения. Задаём формат в промпте.

In [ ]:
%pip install -q langchain langchain-openai python-dotenv
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Извлекай отношения между сущностями из текста. Формат: сущность1 | тип_связи | сущность2. Одна строка на отношение."),
    ("human", "{text}")
])

chain = prompt | llm
text = "Иван Петров работает в компании Яндекс. Его руководитель — директор Мария Сидорова."
result = chain.invoke({"text": text})
print("LLM relation extraction:")
print(result.content)

## 3. Event Extraction с помощью LLM

Событие: действие + участники + время/место. Структурируем через JSON.

In [ ]:
event_prompt = ChatPromptTemplate.from_messages([
    ("system", "Извлеки события из текста. Для каждого события укажи: действие, участники (кто), время (когда), место (где). Ответ в JSON: [{\"action\": \"...\", \"participants\": [...], \"time\": \"...\", \"place\": \"...\"}]"),
    ("human", "{text}")
])

text2 = "15 марта 2024 года в Москве прошла встреча гендиректора Роснефти Игоря Сечина с министром энергетики. Обсуждали поставки нефти."
events = (event_prompt | llm).invoke({"text": text2})
print("Event extraction:")
print(events.content)

## 4. LLM для аннотации (few-shot)

Few-shot помогает задать желаемый формат и стиль аннотации.

In [ ]:
few_shot_prompt = """
Извлеки ключевые факты. Формат: ФАКТ | ИСТОЧНИК (фрагмент текста).

Пример:
Текст: Компания Apple выпустила новый iPhone 16 в сентябре 2024.
Факты:
Apple выпустила iPhone 16 | "выпустила новый iPhone 16"
Дата релиза — сентябрь 2024 | "в сентябре 2024"

Текст: {text}
Факты:
"""

facts = llm.invoke(few_shot_prompt.format(text=text2))
print("Few-shot факты:")
print(facts.content)

## Микро-упражнение

Извлеки отношения из 2–3 своих предложений с помощью LLM. Проверь, что формат соблюдён.